In [ ]:
# # Use the latest version of pip.
# !pip install --upgrade pip
# # Install tfx and kfp Python packages.
# !pip install --upgrade "tfx[kfp]<2"
# !pip install python-dotenv

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

PIPELINE_NAME_BASE = 'advert_pipeline'
PIPELINE_NAME_DAILY = 'advert_pipeline_daily'
PIPELINE_NAME_TUNER = 'advert_pipeline_tuner'
GIT_REPO_NAME = 'mlops-demo'
PROJECT_DIR=os.path.join(os.path.expanduser("~"), GIT_REPO_NAME, 'pipelines', PIPELINE_NAME_BASE)

%cd {PROJECT_DIR}

In [ ]:
import sys
import os
import kfp
from utils import kfp_utils

In [ ]:
# Git에서 제외된 .env 파일 생성 (advert_pipeline 디렉토리)
"""
MYSQL_HOST = 11.333.11.222
MYSQL_PORT = 3306
MYSQL_DATABASE = advert_metadata
MYSQL_USERNAME = root
MYSQL_PASSWORD = abcdefg
LOCAL_METADATA_PATH=/home/.../temp/tfx-output/tfx_metadata/advert_pipeline/metadata.db
"""

In [ ]:
# Kubeflow endpoint
ENDPOINT='https://693bb3e0eb746c18-dot-us-central1.pipelines.googleusercontent.com/' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

In [ ]:
from dotenv import load_dotenv

load_dotenv('.env')  # load secret env

### Tuner Pipeline

In [ ]:
# Create pipeline
!tfx pipeline create --pipeline-path=kubeflow_runner_with_tuner.py --endpoint={ENDPOINT} \
--build-image

In [ ]:
# Run pipeline
!tfx run create --pipeline-name={PIPELINE_NAME_TUNER} --endpoint={ENDPOINT}

In [ ]:
# Update pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner_with_tuner.py \
--endpoint={ENDPOINT} --build-image
# !tfx run create --pipeline-name={PIPELINE_NAME_DAILY} --endpoint={ENDPOINT}

In [ ]:
# Create Recurring run
kfp_utils.create_custom_recurring_runs(host=ENDPOINT, 
                                       experiment_id='31qaq32...',  # kfp client create experiment로 생성 후 list로 id 확인
                                       job_name='hp_tuning', 
                                       description='weekly hyperparameters tuning', 
                                       start_time='2022-05-01T09:00:00Z', 
                                       end_time='2022-05-31T09:00:00Z', 
                                       cron_expression='0 5 0 * * 6',  # 토요일 00:05 마다 run
                                       pipeline_id='23dkf92...',  #  
                                       version_id='51lpe39...', # version이 pipeline보다 우선
                                       no_catchup=False)

### Daily Pipeline

In [ ]:
# Create pipeline
!tfx pipeline create --pipeline-path=kubeflow_runner.py --endpoint={ENDPOINT} \
--build-image

In [ ]:
# Run pipeline
!tfx run create --pipeline-name={PIPELINE_NAME_DAILY} --endpoint={ENDPOINT}

In [ ]:
# Update pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} --build-image
# !tfx run create --pipeline-name={PIPELINE_NAME_DAILY} --endpoint={ENDPOINT}

In [ ]:
# Create Recurring run
kfp_utils.create_custom_recurring_runs(host=ENDPOINT, 
                                       experiment_id='91naq32...',  # kfp client create experiment로 생성 후 list로 id 확인
                                       job_name='daily_training', 
                                       description='daily model training', 
                                       start_time='2022-05-01T09:00:00Z', 
                                       end_time='2022-05-31T09:00:00Z', 
                                       cron_expression='0 5 0 * * 1,2,3,4,5,7',  # 매일 00:05 마다 run (토요일 제외)
                                       pipeline_id='83dkf12...',  #  
                                       version_id='29lue39...', # version이 pipeline보다 우선
                                       no_catchup=False)